### Install pre-requisite packages for the course

Please ensure that this cell is run, to install all pre-requisite packages needed for subsequent code examples.

In [2]:
%pip install python-dotenv
%pip install llama-index
%pip install llama-index-llms-openai
%pip install llama-index-embeddings-openai
%pip install llama-index-llms-azure-openai
%pip install llama-index-embeddings-azure-openai
%pip install llama-index-agent-introspective
%pip install llama-index-program-openai
%pip install llama-index-readers-file
%pip install pydantic
%pip install llama-index-core
%pip install llama-index-utils-workflow
%pip install llama-index-tools-wikipedia
%pip install llama-index-readers-json
%pip install llama-index-readers-file
%pip install pymupdf
%pip install llama-index-tools-google
%pip install nbformat

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

### 01.04. Setup LLM and Tools in LlamaIndex

#### Setup the LLM

In [ ]:
#Using AzureOpenAI as LLM. Other LLM options are available in LlamaIndex
#from llama_index.llms.azure_openai import AzureOpenAI

# Using OpenAI as LLM with LlamaIndex
from llama_index.llms.openai import OpenAI

from llama_index.core import Settings
from dotenv import load_dotenv
load_dotenv()
import os
import nest_asyncio

#Used by LlamaIndex
nest_asyncio.apply()

#API key for OpenAi or AzureOpenAI. Replace with your own key
api_key = os.environ.get("OPENAI_API_KEY")

#Azure Endpoint URL. Replace with your own URL
#azure_endpoint = "https://agentic-ai-course-kumaran.openai.azure.com/"

# Get all models available for my subscription
#import requests
#response = requests.get(
#    "https://api.openai.com/v1/models",
#    headers={"Authorization": f"Bearer {api_key}"}
#)
#print(response.json())

#Note : Function calling support only available in GPT-4+ (maybe also in GPT-3.5 Turbo)
# Create the LLM object for OpenAI
llm = OpenAI(
    model="gpt-4o",  # or "gpt-4-turbo", "gpt-3.5-turbo-16k", etc.
    api_key=api_key
)

#Create the LLM object for Azure OpenAI
#llm=AzureOpenAI(
#    model="gpt-4o",
#    deployment_name="gpt-4o",
#    api_key=api_key,
#    azure_endpoint=azure_endpoint,
#    api_version="2024-08-01-preview",
#)

# Optionally, set as default for LlamaIndex
#Settings.llm = llm

{'object': 'list', 'data': [{'id': 'gpt-4-0613', 'object': 'model', 'created': 1686588896, 'owned_by': 'openai'}, {'id': 'gpt-4', 'object': 'model', 'created': 1687882411, 'owned_by': 'openai'}, {'id': 'gpt-3.5-turbo', 'object': 'model', 'created': 1677610602, 'owned_by': 'openai'}, {'id': 'gpt-4o-audio-preview-2025-06-03', 'object': 'model', 'created': 1748908498, 'owned_by': 'system'}, {'id': 'gpt-4.1-nano-2025-04-14', 'object': 'model', 'created': 1744321025, 'owned_by': 'system'}, {'id': 'gpt-4.1-nano', 'object': 'model', 'created': 1744321707, 'owned_by': 'system'}, {'id': 'gpt-image-1', 'object': 'model', 'created': 1745517030, 'owned_by': 'system'}, {'id': 'gpt-4o-realtime-preview-2025-06-03', 'object': 'model', 'created': 1748907838, 'owned_by': 'system'}, {'id': 'davinci-002', 'object': 'model', 'created': 1692634301, 'owned_by': 'system'}, {'id': 'babbage-002', 'object': 'model', 'created': 1692634615, 'owned_by': 'system'}, {'id': 'gpt-3.5-turbo-instruct', 'object': 'model',

#### Set up Functions

In [12]:
#define a couple of functions
def find_sum(x:int, y:int) -> int :
    #The docstring comment describes the capabilities of the function
    #It is used by the agent to discover the function's inputs, outputs and capabilities
    """
    This function is used to add two numbers and return their sum.
    It takes two integers as inputs and returns an integer as output.
    """
    return x + y

def find_product(x:int, y:int) -> int :
    """
    This function is used to multiply two numbers and return their product.
    It takes two integers as inputs and returns an integer as ouput.
    """
    return x * y

### 01.05. Setup and execute an AI Agent

In [13]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.react.base import ReActAgent

#Create function tools 
sum_tool = FunctionTool.from_defaults(find_sum)
product_tool=FunctionTool.from_defaults(find_product)

#Create a pre-built ReAct agent
simple_agent = ReActAgent.from_tools(
                [sum_tool, product_tool],
                llm=llm,
                verbose=True #Set verbose for detailed logs
                )

#### Test the Agent

In [5]:
response = simple_agent.chat("What is 2 + 2? ")

> Running step 6d4e7545-b12b-42ab-b31a-63acd30d8b43. Step input: What is 2 + 2? 
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: find_sum
Action Input: {'x': 2, 'y': 2}
Observation: 4
> Running step 080e10a0-d62a-4ae0-9a87-6556aaf1fcf5. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 4


In [14]:
response = simple_agent.chat("Multiple 2 by 3 and then add 5")

> Running step e0f532e2-2bba-40e1-8ecc-3818472f9dd8. Step input: Multiple 2 by 3 and then add 5
Thought: The user wants to multiply 2 by 3 and then add 5 to the result. I can use the tools available to help with this calculation.
Action: find_product
Action Input: {'x': 2, 'y': 3}
Observation: 6
> Running step a8352649-0063-46d6-9bca-cdd1a038ce60. Step input: None
Thought: The multiplication of 2 by 3 is 6. Now, I need to add 5 to this result to complete the calculation.
Action: find_sum
Action Input: {'x': 6, 'y': 5}
Observation: 11
> Running step a4425572-576a-445b-bf50-c9af45156225. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The result of multiplying 2 by 3 and then adding 5 is 11.
